In [ ]:
from pathlib import Path
from datetime import timedelta
text = Path("_chat.txt").open("r").read()

In [ ]:
import re
import pandas as pd
from datetime import datetime
from typing import List, Tuple

In [ ]:
def extract_messages(input_text: str) -> List[Tuple[str, str, str]]:
    pattern = re.compile(
        r"\[(?P<date>\d{1,2}\/\d{1,2}\/\d{2,4}), (?P<time>\d{1,2}:\d{2}:\d{2})\] (?P<sender>[^:]+): (?P<message>.+)"
    )
    join_pattern = re.compile(r"joined using this group\'s invite link")

    messages = []
    for line in input_text.split("\n"):
        match = pattern.match(line)
        if match and not join_pattern.search(line):
            date, time, sender, message = match.groups()
            datetime_str = f"{date} {time}"
            dt = datetime.strptime(datetime_str, "%m/%d/%y %H:%M:%S")
            messages.append((sender, dt, message))

    return messages


messages = extract_messages(text)
df = pd.DataFrame(messages, columns=["Sender", "Datetime", "Message"])

print(df)

In [ ]:
def remove_pii(text):
    # Remove phone numbers
    phone_pattern = re.compile(r"@\+?(\d[\d-]{7,}\d)")
    no_phones = phone_pattern.sub("[PHONE REMOVED]", text)

    # Remove email addresses
    email_pattern = re.compile(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b")
    no_emails = email_pattern.sub("[EMAIL REMOVED]", no_phones)

    return no_emails


def cleanup(df):
    # Drop the Sender column
    if "Sender" in df.columns:
        df = df.drop(columns=["Sender"])
    # Drop the rows with no message
    df = df.dropna()

    df = df[~df["Message"].str.contains("deleted this message")]
    df = df[~df["Message"].str.contains("message was deleted")]
    df = df[~df["Message"].str.contains("‎‪")]
    df = df[~df["Message"].str.contains("changed the subject to")]
    df = df[~df["Message"].str.contains("‎")]
    df = df[~df["Message"].str.contains("You added")]
    df = df[~df["Message"].str.contains("changed the group description")]
    df = df[~df["Message"].str.contains("POLL:")]
    df = df[~df["Message"].str.contains("reset this group's invite link")]
    df = df[~df["Message"].str.contains("changed this group's icon")]
    df = df[~df["Message"].str.contains("changed the subject from")]
    df = df[~df["Message"].str.contains("changed this group's settings")]
    df["Message"] = df["Message"].apply(remove_pii)
    return df


print(f"Before cleanup: {len(df)}")

# Export the raw DataFrame to CSV
df.to_csv("raw_latest.csv", index=False)

import pandas as pd
import re

# Parse datetime
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Initialize empty dictionary to keep track of join dates and message counts
join_dates = {}
message_counts = {}

# For each row in the DataFrame
for idx, row in df.iterrows():
    sender = row['Sender']
    message = row['Message']
    datetime = row['Datetime']
    
    # Check if the sender has already joined
    if sender not in join_dates:
        # Check if the message indicates the sender has joined
        if re.search(rf'{re.escape(sender)} (was added|joined via community|joined via an invite link)', message) or re.search(rf'.* added {re.escape(sender)}', message):
            join_dates[sender] = datetime
        else:
            # If the sender is not in the join dates and the message does not indicate they have joined,
            # then this is the first message from the sender and we use the datetime of this message as the join date
            join_dates[sender] = datetime
    
    # Increment the count of messages for the sender
    if sender not in message_counts:
        message_counts[sender] = 1
    else:
        message_counts[sender] += 1

# Create a new DataFrame from the join dates and message counts dictionaries
df_result = pd.DataFrame({
    'Sender': list(join_dates.keys()),
    'Join_Date': list(join_dates.values()),
    'Number_of_Messages': list(message_counts.values())
})

df_result
# df_result[df_result['Sender'] == 'Rahul Bhatnagar']

In [ ]:
# Filter the DataFrame based on the conditions

latest_date = df["Datetime"].max().date().strftime("%Y%m%d")
cutoff_date = pd.to_datetime(latest_date) - timedelta(days=60)
print(f"Latest date: {latest_date}, cutoff date: {cutoff_date}")
df_filtered = df_result[(df_result['Join_Date'] < cutoff_date) & (df_result['Number_of_Messages'] <= 1)]
df_filtered

In [ ]:
# Count the number of senders that contain a "~" in their name
senders_with_tilde = df_filtered[df_filtered['Sender'].str.contains('~')]

# Users to be removed
senders_with_tilde.to_csv("users_to_remove.csv", index=False)
senders_with_tilde

In [ ]:
df_result[df_result['Join_Date'] >= cutoff_date]
# num_users_joined_since_cutoff

In [ ]:
# Today's date
df = cleanup(df)
print(f"After cleanup: {len(df)}")
df.to_csv(f"../{latest_date}_Messages.csv", index=False)